## Imports

In [108]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [109]:
df = pd.read_csv('Bay Area companies that decided to downsize their footprints, Jan. 1, 2022 to July 5, 2022 - Sheet1.csv')

In [110]:
df['Address(es) of space company vacated or plans to vacate, in sf']

0     450 Mission Bay Boulevard and 360 Third Street...
1     690 East Middlefield Road, Mountain View, upon...
2             425 and 525 Market Street, San Francisco 
3                  235 Montgomery Street, San Francisco
4                     270 Brannan Street, San Francisco
5                     1455 Market Street, San Francisco
6     101 Spear Street, San Francisco. Twilio is not...
7                     300 Mission Street, San Francisco
8            2555 and 2565 Augustine Drive, Santa Clara
9                     180 Howard Street, San Francisco 
10                     425 Market Street, San Francisco
11                 101 California Street, San Francisco
12              2975 and 3001 Stender Way, Santa Clara 
13                       150 Rose Orchard Way, San Jose
Name: Address(es) of space company vacated or plans to vacate, in sf, dtype: object

## Image URL collector

In [111]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [112]:
df['image_links'] = ''
links = []
for url in df['Link to story']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [113]:
len(links)

14

In [114]:
df['Company name']

0                 Nektar Therapeutics 
1                            Synopsys 
2                             Sephora 
3               Farella Braun + Martel
4                              Splunk 
5                               Block 
6                               Twilio
7                            Autodesk 
8                       Analog Devices
9              State Bar of California
10                             PayPal 
11    Merrill (formerly Merrill Lynch)
12                             Onsemi 
13                                Arm 
Name: Company name, dtype: object

In [115]:
df["short_description"] = df["Company name"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Company name'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["Link to story"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/sanfrancisco/...
1     <a href="https://therealdeal.com/sanfrancisco/...
2     <a href="https://therealdeal.com/sanfrancisco/...
3     <a href="https://therealdeal.com/sanfrancisco/...
4     <a href="https://therealdeal.com/sanfrancisco/...
5     <a href="https://therealdeal.com/sanfrancisco/...
6     <a href="https://therealdeal.com/sanfrancisco/...
7     <a href="https://therealdeal.com/sanfrancisco/...
8     <a href="https://therealdeal.com/sanfrancisco/...
9     <a href="https://therealdeal.com/sanfrancisco/...
10    <a href="https://therealdeal.com/sanfrancisco/...
11    <a href="https://therealdeal.com/sanfrancisco/...
12    <a href="https://therealdeal.com/sanfrancisco/...
13    <a href="https://therealdeal.com/sanfrancisco/...
Name: description_link, dtype: object

In [116]:
df.description_link[0]

'<a href="https://therealdeal.com/sanfrancisco/2022/04/27/nektar-therapeutics-looks-to-sublease-200k-sf-in-mission-bay/">Nektar Therapeutics</a> '

## Google Maps API Geolocater Setup

In [117]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [118]:
df['geo_address'] = df['Address(es) of space company vacated or plans to vacate, in sf'] + ' CA'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [119]:
df.columns

Index(['Company name', 'Home city', 'Category',
       'Amount of space company vacated or plans to vacate, in sf',
       'Amount of space company leased or subleased during same time period, in sf',
       'Net decline in footprint, in sf',
       'Address(es) of space company vacated or plans to vacate, in sf',
       'Address(es) of space company leased or subleased during same time period, in sf',
       'Date TRD wrote about downsizing plans', 'Link to story', 'image_links',
       'short_description', 'remaining_desc', 'description_link',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [120]:
df

,Company name,Home city,Category,"Amount of space company vacated or plans to vacate, in sf","Amount of space company leased or subleased during same time period, in sf","Net decline in footprint, in sf","Address(es) of space company vacated or plans to vacate, in sf","Address(es) of space company leased or subleased during same time period, in sf",Date TRD wrote about downsizing plans,Link to story,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,Nektar Therapeutics,San Francisco,"Office, life science",Almost 200K sf,NaN,About 200K sf,450 Mission Bay Boulevard and 360 Third Street...,NaN,04.27.2022,https://therealdeal.com/sanfrancisco/2022/04/2...,https://therealdeal.com/sanfrancisco/wp-conten...,Nektar Therapeutics,,"<a href=""https://therealdeal.com/sanfrancisco/...",450 Mission Bay Boulevard and 360 Third Street...,"(360 3rd St, San Francisco, CA 94107, USA, (37...","(37.78267840000001, -122.3983676, 0.0)",37.782678,-122.398368,0.0
1,Synopsys,Sunnyvale,Office,About 170K sf,152K sf,18K sf,"690 East Middlefield Road, Mountain View, upon...","675 Almanor Avenue, Sunnyvale",04.28.2022,https://therealdeal.com/sanfrancisco/2022/04/2...,https://therealdeal.com/sanfrancisco/wp-conten...,Synopsys,,"<a href=""https://therealdeal.com/sanfrancisco/...","690 East Middlefield Road, Mountain View, upon...","(690 E Middlefield Rd, Mountain View, CA 94043...","(37.3945198, -122.0503021, 0.0)",37.394520,-122.050302,0.0
2,Sephora,San Francisco,Office,309K sf,286K sf,23K sf,"425 and 525 Market Street, San Francisco","350 Mission Street, San Francisco",01.14.2022,https://therealdeal.com/sanfrancisco/2022/01/1...,https://therealdeal.com/sanfrancisco/wp-conten...,Sephora,,"<a href=""https://therealdeal.com/sanfrancisco/...","425 and 525 Market Street, San Francisco CA","(First Market Tower, 525 Market St, San Franci...","(37.790507, -122.3990375, 0.0)",37.790507,-122.399038,0.0
3,Farella Braun + Martel,San Francisco,Office,125K sf,40K sf,85K sf,"235 Montgomery Street, San Francisco","1 Bush Street, San Francisco",06.06.2022,https://therealdeal.com/sanfrancisco/2022/06/0...,https://therealdeal.com/sanfrancisco/wp-conten...,Farella Braun +,Martel,"<a href=""https://therealdeal.com/sanfrancisco/...","235 Montgomery Street, San Francisco CA","(Russ Building, 235 Montgomery St, San Francis...","(37.7913674, -122.4028796, 0.0)",37.791367,-122.402880,0.0
4,Splunk,San Francisco,Office,115K sf,NaN,115K sf,"270 Brannan Street, San Francisco",NaN,05.09.2022,https://therealdeal.com/sanfrancisco/2022/05/0...,https://therealdeal.com/sanfrancisco/wp-conten...,Splunk,,"<a href=""https://therealdeal.com/sanfrancisco/...","270 Brannan Street, San Francisco CA","(270 Brannan St, San Francisco, CA 94107, USA,...","(37.7827679, -122.3913735, 0.0)",37.782768,-122.391374,0.0
5,Block,N/A (no HQ address),Office,470K sf,NaN,470K sf,"1455 Market Street, San Francisco",NaN,06.16.2022,https://therealdeal.com/sanfrancisco/2022/06/1...,https://therealdeal.com/sanfrancisco/wp-conten...,Block,,"<a href=""https://therealdeal.com/sanfrancisco/...","1455 Market Street, San Francisco CA","(1455 Market St, San Francisco, CA 94103, USA,...","(37.7749419, -122.417114, 0.0)",37.774942,-122.417114,0.0
6,Twilio,San Francisco,Office,Three floors of offices. The company now occup...,NaN,"Three floors of offices, exact square footage ...","101 Spear Street, San Francisco. Twilio is not...",NaN,06.09.2022,https://therealdeal.com/sanfrancisco/2022/06/0...,https://therealdeal.com/sanfrancisco/wp-conten...,Twilio,,"<a href=""https://therealdeal.com/sanfrancisco/...","101 Spear Street, San Francisco. Twilio is not...","(101 Spear St, San Francisco, CA 94105, USA, (...","(37.792549, -122.3930995, 0.0)",37.792549,-122.393100,0.0
7,Autodesk,San Rafael,Office,117K sf,NaN,117K sf,"300 Mission Street, San Francisco",NaN,01.12.2022,https://therealdeal.com/sanfrancisco/2022/01/1...,https://therealdeal.com/sanfrancisco/wp-conten...,Autodesk,,"<a href=

In [121]:
df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

## HTML popup formatter

In [122]:
df.columns

Index(['Company name', 'Home city', 'Category',
       'Amount of space company vacated or plans to vacate, in sf',
       'Amount of space company leased or subleased during same time period, in sf',
       'Net decline in footprint, in sf',
       'Address(es) of space company vacated or plans to vacate, in sf',
       'Address(es) of space company leased or subleased during same time period, in sf',
       'Date TRD wrote about downsizing plans', 'Link to story', 'image_links',
       'short_description', 'remaining_desc', 'description_link',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [123]:
def popup_html(row):
    i = row
    address = df['Address(es) of space company vacated or plans to vacate, in sf'].iloc[i]
    developer = df['Net decline in footprint, in sf'].iloc[i]
    description = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    sf_plan_vacate = df['Amount of space company vacated or plans to vacate, in sf'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Company: <strong>{}'''.format(description) + '''</strong><br>
    Address(es) vacated or to be vacated: <em>{}'''.format(address) + '''</em><br>
    Amount of space vacated or to be vacated: <strong><em>{}'''.format(sf_plan_vacate) + '''</strong></em><br>
    Net decline in footprint: <strong><em>{}'''.format(developer) + '''</strong></em><br>
    </html>
    '''
    return html


### HTML reservoir

In [124]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [125]:
df.description_link[0]

'<a href="https://therealdeal.com/sanfrancisco/2022/04/27/nektar-therapeutics-looks-to-sublease-200k-sf-in-mission-bay/">Nektar Therapeutics</a> '

In [126]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Bay Area companies that decided to downsize their footprints, January 2022 to July 2022')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [127]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [128]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [129]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/bay_area_company_downsize
